In [1]:
import numpy as np

Solution Function

In [9]:
def solve_linear_equations(coefficients, constants):
    try:
        # Trying to find a unique solution
        solution = np.linalg.solve(coefficients, constants)
    except np.linalg.LinAlgError:
        # Handle overdetermined systems
        solution, _, _, _ = np.linalg.lstsq(coefficients, constants, rcond=None)
    
    return solution


Input Segment

In [3]:
num_variables = int(input("Enter the number of variables (up to 5): "))
num_equations = int(input("Enter the number of equations: "))

In [11]:
coefficients = []
constants = []

for i in range(num_equations):
    equation = input(f"Enter coefficients and constant for equation {i+1}, separated by space: ")
    values = [float(x) for x in equation.split()]
    coefficients.append(values[:-1])  # All but last
    constants.append(values[-1])      # Last element

In [12]:
coefficients_array = np.array(coefficients)
constants_array = np.array(constants)

In [13]:
solution = solve_linear_equations(coefficients_array, constants_array)
print("Solution:", solution)

Solution: [1.71260997 2.60997067 1.52492669]
